In [ ]:
# shakespeare.ipynb
from src.simulate import simulate
from src.helper import display_train_stats, ExperimentLogger, plot_accuracy
from src.config import *

dataset = 'SHAKESPEARE'
img_dir = f'{img_dir_root}/{dataset}'
c, b, e = 10, 50, 5
t_acc = target_accuracy_set[dataset]
label_parameters = f"C={c/100: .2f}, B={b},  E={e}, ACC={t_acc}"
stats, kw = simulate(dataset, num_clients=100, num_participants=c, batch_size=b, num_local_epochs=e, num_rounds=20, learning_rate=0.03, target_accuracy=t_acc, iid=False, mlp=False)
name = f"{dataset}_{kw}_{c}_{b}_{e}"
fig = display_train_stats(stats, title=name, label=label_parameters, target_accuracy=t_acc)
fig.savefig(f'{img_dir}/{kw}_{c}_{b}_{e}.png')

Dataset SHAKESPEARE Loaded!
